In [5]:
#commonly used 4-digit passwords
commonly_used_passwords = {
    '0000', '1111', '2222', '3333', '4444', '5555', '6666', '7777', '8888', '9999', # All repeating digits
    '1234', '2345', '3456', '4567', '5678', '6789', '7890', '8901', '9012', # Ascending sequences
    '4321', '3210', '9876', '8765', '7654', '6543', '5432', '1098', '0987', # Descending sequences
    '1000', '2000', '3000', '4000', '5000', '6000', '7000', '8000', '9000', # Simple patterns
    '0123', '1122', '2211', '1212', '2121', '1313', '3131', '0101', '1010', '0202', '2020',
    '1357', '2468', '0246', '1100', '0011','1230', 
    '0001', '0010', '0100', '1000' 
    '0022', '2200',
    # Years (common for PINs related to dates)
    '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
    '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
    '2019', '2020', '2021', '2022', '2023', '2024', '2025'
}

In [6]:
import datetime

In [7]:
def format_date_part(part):
    return str(part).zfill(2) #formats date to two digits

In [8]:
#this function returns the set of string derived from the demographics given by the user

def generate_demographic_patterns(date_string):
    patterns = set()
    if not date_string:
        return patterns
    try:
        date = datetime.datetime.strptime(date_string, '%Y-%m-%d')
    except ValueError:
        print(f"Warning: Invalid date format for '{date_string}'. Skipping demographic patterns for this date.")
        return patterns

    day = format_date_part(date.day)
    month = format_date_part(date.month)
    year_short = format_date_part(date.year % 100) 

    patterns.add(f"{month}{day}") # MMDD
    patterns.add(f"{day}{month}") # DDMM
    patterns.add(f"{year_short}{month}") # YYMM
    patterns.add(f"{month}{year_short}") # MMYY
    patterns.add(f"{year_short}{day}") # YYDD
    patterns.add(f"{day}{year_short}") # DDYY

    return patterns

In [9]:
#this functiion will check against the demographics provided by the user and also check if it is a commonly used password or not
def check_mpin_strength(mpin, dob_self='', dob_spouse='', anniversary=''):
    reasons = []

    #ensuring it is a 4-digit number
    if not isinstance(mpin, str) or len(mpin) != 4 or not mpin.isdigit():
        print(f"Error: Please enter exactly 4 digits.")
        return {'strength': "INVALID", 'reasons': ['format_is_invalid']}

    # part A: Checking if it is commonly used
    if mpin in commonly_used_passwords:
        reasons.append('COMMONLY_USED')

    # part B: check against demographics of all the three dates
    # Self DOB
    self_dob_patterns = generate_demographic_patterns(dob_self)
    if mpin in self_dob_patterns:
        reasons.append('DEMOGRAPHIC_DOB_SELF')

    # Spouse DOB
    spouse_dob_patterns = generate_demographic_patterns(dob_spouse)
    if mpin in spouse_dob_patterns:
        reasons.append('DEMOGRAPHIC_DOB_SPOUSE')

    # Anniversary
    anniversary_patterns = generate_demographic_patterns(anniversary)
    if mpin in anniversary_patterns:
        reasons.append('DEMOGRAPHIC_ANNIVERSARY')

    strength = "STRONG" if not reasons else "WEAK"
    unique_reasons = sorted(list(set(reasons))) 

    return {'strength': strength, 'reasons': unique_reasons}

In [10]:
#20 test cases
test_cases = [
    # part A: commonly used mpins
    {'mpin': '1111', 'dob_self': '1982-12-31', 'dob_spouse': '1993-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '1234', 'dob_self': '1995-03-15', 'dob_spouse': '1982-07-20', 'anniversary': '2005-11-10', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '0000', 'dob_self': '1982-12-31', 'dob_spouse': '1989-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '9999', 'dob_self': '1995-03-15', 'dob_spouse': '2008-07-20', 'anniversary': '2005-11-10', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '0123', 'dob_self': '1982-12-31', 'dob_spouse': '2003-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '4321', 'dob_self': '1995-03-15', 'dob_spouse': '1998-07-20', 'anniversary': '2005-11-10', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '1122', 'dob_self': '1982-12-31', 'dob_spouse': '1983-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},


    # part B & C: demographic checking
    # self DOB
    {'mpin': '0102', 'dob_self': '1990-02-01', 'dob_spouse': '1982-07-20', 'anniversary': '1982-12-31', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SELF']}}, # DDMM
    {'mpin': '0201', 'dob_self': '1990-01-02', 'dob_spouse': '1998-07-20', 'anniversary': '1995-03-15', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SELF']}}, # MMDD
    {'mpin': '9001', 'dob_self': '1990-01-02', 'dob_spouse': '1982-07-20', 'anniversary': '1982-12-31', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SELF']}}, # YYMM
    {'mpin': '9002', 'dob_self': '1990-02-01', 'dob_spouse': '1998-07-20', 'anniversary': '1995-03-15', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SELF']}}, # YYDD

    # spouse DOB
    {'mpin': '0304', 'dob_self': '1995-03-15', 'dob_spouse': '1985-04-03', 'anniversary': '1998-07-20', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SPOUSE']}},
    {'mpin': '0403', 'dob_self': '1995-03-15', 'dob_spouse': '1985-03-04', 'anniversary': '1998-07-20', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SPOUSE']}},

    # anniversary
    {'mpin': '0506', 'dob_self': '1998-07-20', 'dob_spouse': '1990-01-02', 'anniversary': '2000-06-05', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_ANNIVERSARY']}},
    {'mpin': '0605', 'dob_self': '1990-01-02', 'dob_spouse': '1998-07-20', 'anniversary': '2000-05-06', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_ANNIVERSARY']}},
    
    # strong (not commonly used, no demographic match)
    {'mpin': '7891', 'dob_self': '1982-12-31', 'dob_spouse': '1983-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'STRONG', 'reasons': []}},
    {'mpin': '0167', 'dob_self': '1970-04-05', 'dob_spouse': '1972-06-10', 'anniversary': '1998-09-25', 'expected_output': {'strength': 'STRONG', 'reasons': []}},
    {'mpin': '9871', 'dob_self': '1995-03-15', 'dob_spouse': '1988-07-20', 'anniversary': '2005-11-10', 'expected_output': {'strength': 'STRONG', 'reasons': []}},

    # invalid inputs
    {'mpin': '123', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': {'strength': 'INVALID', 'reasons': ['format_is_invalid']}}, # Invalid length
    {'mpin': 'ABCD', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': {'strength': 'INVALID', 'reasons': ['format_is_invalid']}}, # Non-digit characters
]

In [11]:
#this function runs the above defined cases and shows the expected and actual results
def run_tests():
    for i, test in enumerate(test_cases):
        mpin = test['mpin']
        dob_self = test['dob_self']
        dob_spouse = test['dob_spouse']
        anniversary = test['anniversary']
        expected_output = test['expected_output']
        expected_strength = expected_output['strength']
        expected_reasons = sorted(expected_output['reasons'])

        actual_result = check_mpin_strength(mpin, dob_self, dob_spouse, anniversary)
        actual_strength = actual_result['strength']
        actual_reasons = sorted(actual_result['reasons'])

        passed = (actual_strength == expected_strength) and (actual_reasons == expected_reasons)

        status = "PASSED" if passed else "FAILED"
        print(f"Test {i+1}: MPIN='{mpin}', DOB_Self='{dob_self}', DOB_Spouse='{dob_spouse}', Anniv='{anniversary}'")
        print(f"  Expected: Strength='{expected_strength}', Reasons={expected_reasons}")
        print(f"  Actual:   Strength='{actual_strength}', Reasons={actual_reasons}")
        print(f"  Status: {status}\n")

In [12]:
#Run this block to input dates
mpin = input("Please enter a 4-digit MPIN to check: ")
dob_self = input("Enter your Date of Birth (YYYY-MM-DD): ")
dob_spouse = input("Enter your Spouse's Date of Birth (YYYY-MM-DD): ")
anniversary = input("Enter your Wedding Anniversary (YYYY-MM-DD): ")

result = check_mpin_strength(mpin, dob_self, dob_spouse, anniversary)
print(f"\nYour MPIN '{mpin}' is: {result['strength']}")
if result['reasons']:
    print(f"Reasons: {', '.join(result['reasons'])}")

Please enter a 4-digit MPIN to check:  1708
Enter your Date of Birth (YYYY-MM-DD):  2004-08-17
Enter your Spouse's Date of Birth (YYYY-MM-DD):  2003-03-12
Enter your Wedding Anniversary (YYYY-MM-DD):  2025-06-04



Your MPIN '1708' is: WEAK
Reasons: DEMOGRAPHIC_DOB_SELF


In [13]:
#run all 20 test cases
run_tests()

Test 1: MPIN='1111', DOB_Self='1982-12-31', DOB_Spouse='1993-01-01', Anniv='2001-07-07'
  Expected: Strength='WEAK', Reasons=['COMMONLY_USED']
  Actual:   Strength='WEAK', Reasons=['COMMONLY_USED']
  Status: PASSED

Test 2: MPIN='1234', DOB_Self='1995-03-15', DOB_Spouse='1982-07-20', Anniv='2005-11-10'
  Expected: Strength='WEAK', Reasons=['COMMONLY_USED']
  Actual:   Strength='WEAK', Reasons=['COMMONLY_USED']
  Status: PASSED

Test 3: MPIN='0000', DOB_Self='1982-12-31', DOB_Spouse='1989-01-01', Anniv='2001-07-07'
  Expected: Strength='WEAK', Reasons=['COMMONLY_USED']
  Actual:   Strength='WEAK', Reasons=['COMMONLY_USED']
  Status: PASSED

Test 4: MPIN='9999', DOB_Self='1995-03-15', DOB_Spouse='2008-07-20', Anniv='2005-11-10'
  Expected: Strength='WEAK', Reasons=['COMMONLY_USED']
  Actual:   Strength='WEAK', Reasons=['COMMONLY_USED']
  Status: PASSED

Test 5: MPIN='0123', DOB_Self='1982-12-31', DOB_Spouse='2003-01-01', Anniv='2001-07-07'
  Expected: Strength='WEAK', Reasons=['COMMONLY_

In [14]:
#the above written code successfully classifies if a given mpin is weak or easily guessable because of user demographics or has sufficient strength